# 3. Data Imputation

In this notebook, we conduct data imputation with these methods as listed below:
- Median imputation
- KNN (K=5)
- MiceForest
- GAIN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import miceforest as mf
from tqdm import tqdm
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
df = pd.read_csv('../database/companies/original/2016-2023.csv')

/var/folders/kw/6hnl881s2p9701b42gpjt_pr0000gn/T/ipykernel_20460/2360894533.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../database/companies/original/2016-2023.csv')


In [3]:
governance = ["ceo_is_female","unequal_voting","ceo_tenure","board_size","classified_board_system","poison_pill","buyback_yield",
              "dividend_payout_ratio","cf_to_total_compensation_to_executives","cf_to_total_compensation_to_board_members"]

operation = ["cf_to_capex_industry_peers_percentile","net_debt_to_ebitda_industry_peers_percentile",
             "current_ratio_industry_peers_percentile","ebitda_margin_industry_peers_percentile",
             "sales_to_total_assets_industry_peers_percentile","employee_growth_rate_industry_peers_percentile",
             "fcf_yield_industry_peers_percentile","sales_growth_rate_industry_peers_percentile",
             "cash_conversion_cycle_industry_peers_percentile","interest_coverage_ratio_industry_peers_percentile"]

ownership = ["free_float_percentage","institution_ownership_percentage","insider_shares_percentage"]

technical = ["rsi_14_30_average","volatility_30_90_180_average","volume_30d_average_to_outstanding"]

returns = ["total_return_5y_4y_3y_2y_average","total_return_1y_6m_3m_average"]

valuation = ["roe_industry_peers_percentile","operating_roic_industry_peers_percentile","pe_ratio_industry_peers_percentile",
             "eps_industry_peers_percentile","ev_to_sales_industry_peers_percentile","tobin_q_ratio_industry_peers_percentile",
             "pb_ratio_industry_peers_percentile","asset_to_equity_industry_peers_percentile","ev_ebitda_industry_peers_percentile","ev_to_asset_industry_peers_percentile"]


In [4]:
operation_original = ["cf_to_capex","net_debt_to_ebitda","current_ratio","ebitda_margin","sales_to_total_assets",
                      "employee_growth_rate","fcf_yield","sales_growth_rate","cash_conversion_cycle","interest_coverage_ratio"]

technical_original = ['rsi_14d','rsi_30d','volatility_30d','volatility_90d','volatility_180d',"volume_30d_average_to_outstanding"]

returns_original = ['total_return_5y', 'total_return_4y', 'total_return_3y','total_return_2y', 'total_return_1y', 'total_return_6m','total_return_3m']

valuation_original = ["roe","operating_roic","pe_ratio","eps","ev_to_sales","tobin_q_ratio","pb_ratio","asset_to_equity","ev_ebitda","ev_to_asset"]

In [5]:
df["bic_level_2"] = df["bic_level_2"].astype('category')
df["bic_level_3"] = df["bic_level_3"].astype('category')

Each group, from group_1 to group_6, undergoes imputation collectively along with the one-hot encoded year column. 
In contrast, every column within the median_group is subjected to median imputation individually.

In [6]:
median_group = ['ceo_is_female','unequal_voting', 'ceo_tenure', 'board_size', 'classified_board_system', 'poison_pill']
group_1 = ['buyback_yield', 'dividend_payout_ratio', 'cf_to_total_compensation_to_executives', 'cf_to_total_compensation_to_board_members']
group_2 = operation_original
group_3 = ownership
group_4 = technical_original
group_5 = returns_original
group_6 = valuation_original
year_one_hot_encoding = ['year_2016','year_2017','year_2018','year_2019','year_2020','year_2021','year_2022']

In [7]:
def compute_percentile_combined(df, col):
    def compute_percentile(group):
        # Return percentile rank or NaN for small groups
        return group.rank(pct=True) * 100 if len(group) >= 10 else pd.Series(np.nan, index=group.index)

    # Compute percentile based on bic_level_3
    level_3_percentile = df.groupby(['year', 'bic_level_3'], observed=False)[col].transform(compute_percentile)
    
    # Where BIC Level 3 groups are smaller than 10, fallback to BIC Level 2
    mask_fallback = level_3_percentile.isna()
    level_2_percentile = df.groupby(['year', 'bic_level_2'], observed=False)[col].transform(compute_percentile)
    level_3_percentile[mask_fallback] = level_2_percentile[mask_fallback]

    return level_3_percentile

## A. Median Imputation
- For features end with industry_peer_percentile_values, we impute 50. 
- For the rest, we group by the year and impute the median value. Same for the binary values

In [8]:
df_median = df.copy()

median_columns = governance + ownership + technical_original + returns_original
constant_50_columns = operation + valuation

# Impute median for selected columns grouped by 'year' in the copy
df_median[median_columns] = df_median.groupby('year')[median_columns].transform(lambda x: x.fillna(x.median()))

# Impute 50 for selected columns in the copy
df_median[constant_50_columns] = df_median[constant_50_columns].fillna(50)

In [9]:
df_median.to_csv('../database/companies/imputation/median/median_original.csv')

In [10]:
df = df[df['year']!=2023]

## B. KNN Imputation (K = 5)

In [11]:
df_knn = df.copy()

# Function to perform KNN Imputation
def knn_impute(df, group_cols, year_cols):
    imputer = KNNImputer(n_neighbors=5)
    imputed_data = imputer.fit_transform(df[group_cols + year_cols])
    return pd.DataFrame(imputed_data, columns=group_cols + year_cols, index=df.index)

# Perform KNN Imputation for each group
df_knn[group_1 + year_one_hot_encoding] = knn_impute(df_knn, group_1, year_one_hot_encoding)
df_knn[group_2 + year_one_hot_encoding] = knn_impute(df_knn, group_2, year_one_hot_encoding)
df_knn[group_3 + year_one_hot_encoding] = knn_impute(df_knn, group_3, year_one_hot_encoding)
df_knn[group_4 + year_one_hot_encoding] = knn_impute(df_knn, group_4, year_one_hot_encoding)
df_knn[group_5 + year_one_hot_encoding] = knn_impute(df_knn, group_5, year_one_hot_encoding)
df_knn[group_6 + year_one_hot_encoding] = knn_impute(df_knn, group_6, year_one_hot_encoding)

# Median Imputation for median_group
df_knn[median_group] = df_median.groupby('year')[median_group].transform(lambda x: x.fillna(x.median()))

In [12]:
df_knn['rsi_14_30_average'] = df_knn[['rsi_14d','rsi_30d']].mean(axis=1, skipna=True)
df_knn['volatility_30_90_180_average'] = df_knn[['volatility_30d','volatility_90d','volatility_180d']].mean(axis=1, skipna=True)

df_knn['total_return_5y_4y_3y_2y_average'] = df_knn[['total_return_5y', 'total_return_4y', 'total_return_3y', 'total_return_2y']].mean(axis=1, skipna=True)
df_knn['total_return_1y_6m_3m_average'] = df_knn[['total_return_1y', 'total_return_6m', 'total_return_3m']].mean(axis=1, skipna=True)

for col in operation_original + valuation_original:
    percentile_col = col + '_industry_peers_percentile'
    df_knn[percentile_col] = compute_percentile_combined(df_knn, col)
    df_knn[percentile_col] = df_knn[percentile_col].astype(float)

In [13]:
df_knn.to_csv('../database/companies/imputation/kNN/kNN_original.csv')

## C. MiceForest Imputation

In [14]:
# Make a copy of the original dataframe
df_miceforest = df.copy()

# List of groups
groups = [group_1, group_2, group_3, group_4, group_5, group_6]

# Perform imputation for each group
for group in groups:
    # Creating a kernel only with the columns in the current group
    kernel = mf.ImputationKernel(df_miceforest[group+year_one_hot_encoding], save_all_iterations=True, random_state=1991)

    # Running the MICE algorithm
    kernel.mice(3)  # Number of iterations

    # Updating the columns in the original dataset
    df_miceforest[group] = kernel.complete_data()[group]

# Median Imputation for median_group
df_miceforest[median_group] = df_miceforest.groupby('year')[median_group].transform(lambda x: x.fillna(x.median()))

In [15]:
df_miceforest['rsi_14_30_average'] = df_miceforest[['rsi_14d','rsi_30d']].mean(axis=1, skipna=True)
df_miceforest['volatility_30_90_180_average'] = df_miceforest[['volatility_30d','volatility_90d','volatility_180d']].mean(axis=1, skipna=True)

df_miceforest['total_return_5y_4y_3y_2y_average'] = df_miceforest[['total_return_5y', 'total_return_4y', 'total_return_3y', 'total_return_2y']].mean(axis=1, skipna=True)
df_miceforest['total_return_1y_6m_3m_average'] = df_miceforest[['total_return_1y', 'total_return_6m', 'total_return_3m']].mean(axis=1, skipna=True)

for col in operation_original + valuation_original:
    percentile_col = col + '_industry_peers_percentile'
    df_miceforest[percentile_col] = compute_percentile_combined(df_miceforest, col)
    df_miceforest[percentile_col] = df_miceforest[percentile_col].astype(float)


In [16]:
df_miceforest.to_csv('../database/companies/imputation/MiceForest/MiceForest_original.csv')

## D. GAIN Imputation

[Reference: Imputation with GAIN](https://www.kaggle.com/code/youseefmoemen/generative-adversarial-imputation-network-gain)

In [17]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()
  
  if parameters is None:
  
    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
    
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)   
      
    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}

  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']
    
    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)  
      
    norm_parameters = parameters    
      
  return norm_data, norm_parameters


def renormalization (norm_data, norm_parameters):
  '''Renormalize data from [0, 1] range to the original range.
  
  Args:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  
  Returns:
    - renorm_data: renormalized original data
  '''
  
  min_val = norm_parameters['min_val']
  max_val = norm_parameters['max_val']

  _, dim = norm_data.shape
  renorm_data = norm_data.copy()
    
  for i in range(dim):
    renorm_data[:,i] = renorm_data[:,i] * (max_val[i] + 1e-6)   
    renorm_data[:,i] = renorm_data[:,i] + min_val[i]
    
  return renorm_data


def rounding (imputed_data, data_x):
  '''Round imputed data for categorical variables.
  
  Args:
    - imputed_data: imputed data
    - data_x: original data with missing values
    
  Returns:
    - rounded_data: rounded imputed data
  '''
  
  _, dim = data_x.shape
  rounded_data = imputed_data.copy()
  
  for i in range(dim):
    temp = data_x[~np.isnan(data_x[:, i]), i]
    # Only for the categorical variable
    if len(np.unique(temp)) < 20:
      rounded_data[:, i] = np.round(rounded_data[:, i])
      
  return rounded_data


def rmse_loss (ori_data, imputed_data, data_m):
  '''Compute RMSE loss between ori_data and imputed_data
  
  Args:
    - ori_data: original data without missing values
    - imputed_data: imputed data
    - data_m: indicator matrix for missingness
    
  Returns:
    - rmse: Root Mean Squared Error
  '''
  
  ori_data, norm_parameters = normalization(ori_data)
  imputed_data, _ = normalization(imputed_data, norm_parameters)
    
  # Only for missing values
  nominator = np.sum(((1-data_m) * ori_data - (1-data_m) * imputed_data)**2)
  denominator = np.sum(1-data_m)
  
  rmse = np.sqrt(nominator/float(denominator))
  
  return rmse


def xavier_init(size):
  '''Xavier initialization.
  
  Args:
    - size: vector size
    
  Returns:
    - initialized random vector.
  '''
  in_dim = size[0]
  xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
  return tf.random_normal(shape = size, stddev = xavier_stddev)
      

def binary_sampler(p, rows, cols):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix


def uniform_sampler(low, high, rows, cols):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = [rows, cols])       


def sample_batch_index(total, batch_size):
  '''Sample index of the mini-batch.
  
  Args:
    - total: total number of samples
    - batch_size: batch size
    
  Returns:
    - batch_idx: batch index
  '''
  total_idx = np.random.permutation(total)
  batch_idx = total_idx[:batch_size]
  return batch_idx

In [18]:
def gain (data_x, gain_parameters):
  '''Impute missing values in data_x
  
  Args:
    - data_x: original data with missing values
    - gain_parameters: GAIN network parameters:
      - batch_size: Batch size
      - hint_rate: Hint rate
      - alpha: Hyperparameter
      - iterations: Iterations
      
  Returns:
    - imputed_data: imputed data
  '''
  # Define mask matrix
  data_m = 1-np.isnan(data_x)
  
  # System parameters
  batch_size = gain_parameters['batch_size']
  hint_rate = gain_parameters['hint_rate']
  alpha = gain_parameters['alpha']
  iterations = gain_parameters['iterations']
  
  # Other parameters
  no, dim = data_x.shape
  
  # Hidden state dimensions
  h_dim = int(dim)
  
  # Normalization
  norm_data, norm_parameters = normalization(data_x)
  norm_data_x = np.nan_to_num(norm_data, 0)
  
  ## GAIN architecture   
  # Input placeholders
  # Data vector
  X = tf.placeholder(tf.float32, shape = [None, dim])
  # Mask vector 
  M = tf.placeholder(tf.float32, shape = [None, dim])
  # Hint vector
  H = tf.placeholder(tf.float32, shape = [None, dim])
  
  # Discriminator variables
  D_W1 = tf.Variable(xavier_init([dim*2, h_dim])) # Data + Hint as inputs
  D_b1 = tf.Variable(tf.zeros(shape = [h_dim]))
  
  D_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
  D_b2 = tf.Variable(tf.zeros(shape = [h_dim]))
  
  D_W3 = tf.Variable(xavier_init([h_dim, dim]))
  D_b3 = tf.Variable(tf.zeros(shape = [dim]))  # Multi-variate outputs
  
  theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]
  
  #Generator variables
  # Data + Mask as inputs (Random noise is in missing components)
  G_W1 = tf.Variable(xavier_init([dim*2, h_dim]))  
  G_b1 = tf.Variable(tf.zeros(shape = [h_dim]))
  
  G_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
  G_b2 = tf.Variable(tf.zeros(shape = [h_dim]))
  
  G_W3 = tf.Variable(xavier_init([h_dim, dim]))
  G_b3 = tf.Variable(tf.zeros(shape = [dim]))
  
  theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]
  
  ## GAIN functions
  # Generator
  def generator(x,m):
    # Concatenate Mask and Data
    inputs = tf.concat(values = [x, m], axis = 1) 
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)   
    # MinMax normalized output
    G_prob = tf.nn.sigmoid(tf.matmul(G_h2, G_W3) + G_b3) 
    return G_prob
      
  # Discriminator
  def discriminator(x, h):
    # Concatenate Data and Hint
    inputs = tf.concat(values = [x, h], axis = 1) 
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)  
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    D_logit = tf.matmul(D_h2, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob
  
  ## GAIN structure
  # Generator
  G_sample = generator(X, M)
 
  # Combine with observed data
  Hat_X = X * M + G_sample * (1-M)
  
  # Discriminator
  D_prob = discriminator(Hat_X, H)
  
  ## GAIN loss
  D_loss_temp = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) \
                                + (1-M) * tf.log(1. - D_prob + 1e-8)) 
  
  G_loss_temp = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
  
  MSE_loss = \
  tf.reduce_mean((M * X - M * G_sample)**2) / tf.reduce_mean(M)
  
  D_loss = D_loss_temp
  G_loss = G_loss_temp + alpha * MSE_loss 
  
  ## GAIN solver
  D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
  G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
  
  ## Iterations
  sess = tf.Session()
  sess.run(tf.global_variables_initializer())
   
  # Start Iterations
  for it in tqdm(range(iterations)):    
      
    # Sample batch
    batch_idx = sample_batch_index(no, batch_size)
    X_mb = norm_data_x[batch_idx, :]  
    M_mb = data_m[batch_idx, :]  
    # Sample random vectors  
    Z_mb = uniform_sampler(0, 0.01, batch_size, dim) 
    # Sample hint vectors
    H_mb_temp = binary_sampler(hint_rate, batch_size, dim)
    H_mb = M_mb * H_mb_temp
      
    # Combine random vectors with observed vectors
    X_mb = M_mb * X_mb + (1-M_mb) * Z_mb 
      
    _, D_loss_curr = sess.run([D_solver, D_loss_temp], 
                              feed_dict = {M: M_mb, X: X_mb, H: H_mb})
    _, G_loss_curr, MSE_loss_curr = \
    sess.run([G_solver, G_loss_temp, MSE_loss],
             feed_dict = {X: X_mb, M: M_mb, H: H_mb})
            
  ## Return imputed data      
  Z_mb = uniform_sampler(0, 0.01, no, dim) 
  M_mb = data_m
  X_mb = norm_data_x          
  X_mb = M_mb * X_mb + (1-M_mb) * Z_mb 
      
  imputed_data = sess.run([G_sample], feed_dict = {X: X_mb, M: M_mb})[0]
  
  imputed_data = data_m * norm_data_x + (1-data_m) * imputed_data
  
  # Renormalization
  imputed_data = renormalization(imputed_data, norm_parameters)  
  
  # Rounding
  imputed_data = rounding(imputed_data, data_x)  
          
  return imputed_data

In [19]:
df_GAIN = df.copy()

gain_parameters = {
    'batch_size': 128,
    'hint_rate': 0.9,
    'alpha': 10,
    'iterations': 1000
}

groups = [group_1, group_2, group_3, group_4, group_5, group_6]

for group in groups:
    data_x = df_GAIN[group].values
    imputed_data = gain(data_x, gain_parameters)
    df_GAIN[group] = np.where(np.isnan(df_GAIN[group]), imputed_data, df_GAIN[group])


2024-04-23 17:32:59.016411: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 1485.88it/s]


In [20]:
df_GAIN[median_group] = df_GAIN.groupby('year')[median_group].transform(lambda x: x.fillna(x.median()))

df_GAIN['rsi_14_30_average'] = df_GAIN[['rsi_14d','rsi_30d']].mean(axis=1, skipna=True)
df_GAIN['volatility_30_90_180_average'] = df_GAIN[['volatility_30d','volatility_90d','volatility_180d']].mean(axis=1, skipna=True)

df_GAIN['total_return_5y_4y_3y_2y_average'] = df_GAIN[['total_return_5y', 'total_return_4y', 'total_return_3y', 'total_return_2y']].mean(axis=1, skipna=True)
df_GAIN['total_return_1y_6m_3m_average'] = df_GAIN[['total_return_1y', 'total_return_6m', 'total_return_3m']].mean(axis=1, skipna=True)

for col in operation_original + valuation_original:
    percentile_col = col + '_industry_peers_percentile'
    df_GAIN[percentile_col] = compute_percentile_combined(df_miceforest, col)
    df_GAIN[percentile_col] = df_GAIN[percentile_col].astype(float)


In [21]:
df_GAIN.to_csv('../database/companies/imputation/GAIN/GAIN_original.csv')